### Relax and hold steady

***Le but de ce notebook est d'implémenter le flux de Stokes dans une cavité carrée, en résolvant une équation biharmonique.***

Le flux de Stokes est très pratique, car il permet de simplifier l'équation de Navier-Stokes en éliminant la non-linéarité. Après un changement d'échelle, l'équation de Navier-Stokes s'écrit:

\begin{equation}
Re \left(\frac{\partial u^*}{\partial t} + u^* \cdot \nabla u^* \right) = -\nabla p^* + \nabla^2 u^*
\end{equation}
 Dans le flux de Stokes, on suppose que le nombre de Reynolds tend vers zero, ce qui va simplifier cette équation.Prenons le rotational après simplification:
\begin{equation}
\nabla ^2 \omega = 0
\end{equation}

où $\nabla \times u = \omega$ est la vorticité.  

Regardons ce qu'il en est de la fonction de "stream":

\begin{equation}
u = \frac{\partial \psi}{\partial y} \text{   and   } v = - \frac{\partial \psi}{\partial x}
\end{equation}

et

\begin{equation}
\omega = \frac{\partial v}{\partial x} - \frac{\partial u}{\partial y}
\end{equation}
 
Finalement en combinant les trois équations, on obtient: 

\begin{equation}
\nabla^4 \psi= 0
\end{equation}
 

Il nous font donc discrétiser cette équation, pour cela nous allons faire une discrétisation centrale finie:
*Wikipedia (https://fr.wikipedia.org/wiki/%C3%89quation_biharmonique)*

$$\nabla ^4 \psi = \frac{\partial^4 \psi}{\partial x^4}+\frac{\partial^4 \psi}{\partial y^4}+2\frac{\partial^4 \psi}{\partial x^2 \partial y^2}=0$$ 


\begin{array}{rcccl}
\frac{\partial^2\frac{\partial^2 \psi}{\partial x^2}}{\partial x^2}& \Rightarrow &\frac{\left(\frac{\partial^2 \psi}{\partial x^2} \right)_{i+1,j}-2\left(\frac{\partial^2 \psi}{\partial x^2} \right)_{i,j}+\left(\frac{\partial^2 \psi}{\partial x^2} \right)_{i-1,j}}{\Delta x^2}&=&\frac{\left(\psi_{i+1,j}-2\psi_{i,j}+\psi_{i-1,j} \right)_{i+1,j}-2\left(\psi_{i+1,j}-2\psi_{i,j}+\psi_{i-1,j} \right)_{i,j}+\left(\psi_{i+1,j}-2\psi_{i,j}+\psi_{i-1,j} \right)_{i-1,j}}{\Delta x^4}\\
& & &=&\frac{\psi_{i+2,j}-2\psi_{i+1,j}+\psi_{i,j}-2(\psi_{i+1,j}-2\psi_{i,j}+\psi_{i-1,j})+\psi_{i,j}-2\psi_{i-1,j}+\psi_{i-2,j}}{\Delta x^4}
\end{array}

En procédans de même pour les deux autres termes et en considérans que $\Delta x= \Delta y$, on obtien finalement:

$$\psi_{i,j}^k = \frac{1}{20}\left( 8(\psi_{i+1,j}^k+\psi_{i,j+1}^k+\psi_{i-1,j}^k+\psi_{i,j-1}^k)-2(\psi_{i+1,j-1}^k+\psi_{i+1,j+1}^k+\psi_{i-1,j-1}^k)-(\psi_{i+2,j}^k+\psi_{i,j+2}^k+\psi_{i-2,j}^k+\psi_{i,j-2}^k)\right)$$

Nous allons en particulier nous intéresser au problème du *lid-driven cavity flow*, où notre carré possède un couvercle qui va se déplacer à vitesse constante $u = 1$. Et où il n'y a aucun fluide qui peut sortir. Nous allons aussi supposer que toutes les surfaces, le couvercle compris, possède des conditions aux bords de "no-slip".

Jettons un coup d'oeil aux conditions aux bords:

<img src="./figures/drivencavity.svg" width=400px>

**Conditions aux bords**


en $x=0$ on aura $\frac{\partial \psi}{\partial x}=0$
On utilise la forwar 2nd order diff:
$$\frac{-3/2 \psi_{i,j}+2\psi_{i+1,j}-1/2\psi_{i+2,j}}{\Delta x}=0$$

Or on sait aussi que $\psi$ s'annule sur le bord et donc $\psi_{i,j}=0$. $\Rightarrow \psi_{i+1,j}=1/4 \psi_{i+2,j}$ 

en $x=l$ on devra utilisé la backwards diff:
$$\psi_{i-1,j}=1/4\psi_{i-2,j}$$

en $y=0$ on utilise la forward:

$$\psi_{i,j+1}=1/4\psi_{i,j+2}$$

en y=l on aura $\frac{\partial \psi}{\partial y}=1$
$$-2\frac{2\psi_{i,-1}+1/2\psi_{i,-2}}{\Delta y}=1 \Leftrightarrow \psi_{i,l-1}=1/2(1/2\psi_{i,l-2}-\Delta y)$$

In [13]:
import numpy as np
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams, cm
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
import matplotlib.animation as animation
from IPython.display import HTML

from laplace_helper import L2_rel_error

In [14]:
nx = 41
ny = 41
L  =  1
H =  1

dx  =  L/(nx-1)
dy=  H/(ny-1)

l2_norm = 1
l1_target = 1e-8

p_i = np.zeros((ny,nx))

In [15]:
#l=ind de la position en l
l=int(L/dx)   #dx=dy
delta_x=dx
def border_cond(l, delta_x):
    #neumann cond
    psi[1:-1,1] = 1/4*psi[1:-1,2]     #en x = 0 (forward 2nd order)
    psi[1:-1,l-1] = 1/4*psi[1:-1,l-2]   #en x = l
    psi[1,1:-1]= 1/4*psi[2,1:-1]         #en y=0
    psi[l-1,1:-1]=1/2*(1/2*psi[l-2,1:-1]-delta_x)     #en y = L

In [16]:
def conjugate_gradient_biharm(p, l, delta_x, dx, dy, l2_target):
    '''Performs cg relaxation
    Assumes Dirichlet boundary conditions p=0
    
    Parameters:
    ----------
    p : 2D array of floats
        Initial guess
    b : 2D array of floats
        Source term
    dx: float
        Mesh spacing in x direction
    dy: float
        Mesh spacing in y direction
    l2_target: float
        exit criterion
        
    Returns:
    -------
    p: 2D array of float
        Distribution after relaxation
    '''
    ny, nx = p.shape
    r  = np.zeros((ny,nx)) # residual
    Ad  = np.zeros((ny,nx)) # to store result of matrix multiplication 
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # Step-0 We compute the initial residual and 
    # the first search direction is just this residual

    r[1:-2,1:-2]= 1/20 * (8*(p[1:-2,2:-1] + p[2:-1,1:-2] + p[1:-2,1:-2]+ p[1:-2, 1:-2])\
                          -2*(p[1:-2,2:-1]+p[2:-1,2:-1]+p[1:-2,1:-2])\
                          -(p[1:-2,3:]+p[3:,1:-2]+p[1:-2,:-3]+p[:-3,1:-2]))
    
    
    #r[1:-1,1:-1] = 4*p[1:-1,1:-1] - \
        #p[1:-1,2:] - p[1:-1,:-2] - p[2:, 1:-1] - p[:-2, 1:-1]
    d = r.copy()
    rho = np.sum(r*r)
    
    Ad[1:-2,1:-2]= -1/20 * (8*(d[1:-2,2:-1] - d[2:-1,1:-2] - d[1:-2,1:-2]- \
                        d[1:-2, 1:-2])+2*(d[1:-2,2:-1]-d[2:-1,2:-1]-\
                        d[1:-2,1:-2])+(d[1:-2,3:]-d[3:,1:-2]-\
                        d[1:-2,:-3]-d[:-3,1:-2]))
    
    #Ad[1:-1,1:-1] = -4*d[1:-1,1:-1]+d[1:-1,2:]+d[1:-1,:-2]+\
        #d[2:, 1:-1] + d[:-2, 1:-1]
    sigma = np.sum(d*Ad)
    
    # Iterations
    while l2_norm > l2_target:

        pk = p.copy()
        rk = r.copy()
        dk = d.copy()
        
        alpha = rho/sigma

        p = pk + alpha*dk
        r = rk- alpha*Ad
        
        rhop1 = np.sum(r*r)
        beta = rhop1 / rho
        rho = rhop1
        
        d = r + beta*dk
        Ad[1:-2,1:-2]= -1/20 * (8*(d[1:-2,2:-1] - d[2:-1,1:-2] - d[1:-2,1:-2]- \
                        d[1:-2, 1:-2])+2*(d[1:-2,2:-1]-d[2:-1,2:-1]-\
                        d[1:-2,1:-2])+(d[1:-2,3:]-d[3:,1:-2]-\
                        d[1:-2,:-3]-d[:-3,1:-2]))
       # Ad[1:-1,1:-1] = -4*d[1:-1,1:-1] + d[1:-1,2:] + d[1:-1,:-2] + \
         #   d[2:, 1:-1] + d[:-2, 1:-1]
        sigma = np.sum(d*Ad)
        
        # BCs are automatically enforced
        #p=border_cond(l, delta_x)
        
        l2_norm = L2_rel_error(pk,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of CG iterations: {0:d}'.format(iterations))
    return p, l2_conv     


In [17]:
p, l2_conv = conjugate_gradient_biharm(p_i.copy(), l, delta_x, dx, dy, l1_target)
#L2_rel_error(p,pan)

Number of CG iterations: 1


/home/noemie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in double_scalars
